In [9]:
import os
import h5py
import pandas as pd

# Specify the directory containing the .h5 files
directory_path = r'C:\Users\hs\Documents\AnyBody models\AMMR4-Beta\Application\Validation\EvaluateMomentArms\Output'

def traverse_h5_group(group, path='', angular_velocity=None, moment_arm_label='MomentArm_Flx'):
    data = []

    for key in group.keys():
        item = group[key]
        new_path = f"{path}.{key}" if path else key
        if isinstance(item, h5py.Group):
            data.extend(traverse_h5_group(item, new_path, angular_velocity, moment_arm_label))
        elif isinstance(item, h5py.Dataset):
            if key == 'Strength':
                muscle_name = new_path.replace('.Strength', '') + ';'
                strength_array = item[()]
                strength = strength_array[0] if strength_array.size > 0 else None
                lmt_dot_array = group['LmtDot'][()] if 'LmtDot' in group else None
                entry = {'MuscleName': muscle_name, 'Strength': strength}

                if lmt_dot_array is not None and angular_velocity is not None:
                    for i, lmt_dot in enumerate(lmt_dot_array):
                        entry[f'{moment_arm_label}_{i}'] = -lmt_dot / angular_velocity if angular_velocity != 0 else None
                
                data.append(entry)

    return data

def get_angular_velocity(file_path):
    try:
        with h5py.File(file_path, 'r') as h5_file:
            angular_velocity_path = 'Output/AngularVelocity_output/Val'
            if angular_velocity_path in h5_file:
                angular_velocity_dataset = h5_file[angular_velocity_path]
                angular_velocity = angular_velocity_dataset[0] if angular_velocity_dataset.shape else angular_velocity_dataset[()]
                return angular_velocity
    except Exception as e:
        print(f"Error reading angular velocity from {file_path}: {e}")
    return None

def process_h5_file(file_path, moment_arm_label):
    angular_velocity = get_angular_velocity(file_path)

    with h5py.File(file_path, 'r') as h5_file:
        try:
            base_group = h5_file['Output']['_Main']['HumanModel']['BodyModel']['Trunk']['Muscles']
        except KeyError as e:
            print(f"KeyError: {e}. The file structure is not as expected.")
            return pd.DataFrame()

        muscles_data = traverse_h5_group(base_group, angular_velocity=angular_velocity, moment_arm_label=moment_arm_label)

    df = pd.DataFrame(muscles_data)
    df.dropna(axis=1, how='all', inplace=True)
    moment_arm_cols = [col for col in df.columns if col.startswith('MomentArm_')]
    for col in moment_arm_cols:
        df = df[df[col].apply(lambda x: x is not None and abs(x) >= 1e-6)]

    return df

def process_all_h5_files(directory):
    all_data = {}

    for filename in os.listdir(directory):
        if filename.endswith('.h5'):
            file_path = os.path.join(directory, filename)
            parts = filename.split('_')
            joint_name = parts[-2]
            movement_type = parts[-1].replace('.h5', '')

            if 'Extension' in movement_type:
                df = process_h5_file(file_path, 'MomentArm_Flx')
            elif 'LateralBending' in movement_type:
                df = process_h5_file(file_path, 'MomentArm_Lat')
            elif 'AxialRotation' in movement_type:
                df = process_h5_file(file_path, 'MomentArm_Axial')
            else:
                continue
            
            if not df.empty:
                if joint_name in all_data:
                    all_data[joint_name] = pd.merge(all_data[joint_name], df, on=['MuscleName', 'Strength'], how='outer', suffixes=('', '_new'))
                    for col in all_data[joint_name].columns:
                        if col.endswith('_new'):
                            base_col = col[:-4]
                            all_data[joint_name][base_col] = all_data[joint_name].apply(lambda row: row[base_col] if pd.notna(row[base_col]) else row[col], axis=1)
                            all_data[joint_name].drop(columns=col, inplace=True)
                else:
                    all_data[joint_name] = df

    with pd.ExcelWriter('muscle_MomentArm.xlsx', engine='openpyxl') as writer:
        for joint_name, df in all_data.items():
            if not df.empty:
                df.to_excel(writer, sheet_name=joint_name, index=False)
                print(f"Data written to sheet '{joint_name}'")
            else:
                print(f"No valid data for joint '{joint_name}', skipping.")

# Process all HDF5 files in the specified directory
process_all_h5_files(directory_path)


Data written to sheet 'Trunk'
